In [3]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "


# function to preprocess movie reviews such as tokenization, removing stopwords, and removing punctuations.
# also, this function reads posivite and negative reviews from separate files and combile them into 
# a single file with their class labels.

def preprocessing():
    import pandas as pd 
    from pandas import DataFrame, read_csv
    import os
    import csv 
    import numpy as np 

    # read stop words
    stopwords = open("/stopwords.txt", 'r' ).read()
    stopwords = stopwords.split("\n")

    indices = []
    text = [] # reviews
    rating = [] # sentiment of a reviews: 0 -> posivite, 1-> negative

    i =  0 

    # read dataset
    # read positive reviews 
    pos_lines = open("/dataset/train/imdb_train_pos.txt", 'r' ).read()
    plines = pos_lines.split("\n")   
    plines = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in plines]
    plines = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in plines]
    
    for line in plines:
        data = remove_stopwords(line, stopwords)
        indices.append(i)
        text.append(data)
        rating.append("1")
        i = i + 1

    # read negative reviews
    neg_lines = open("/dataset/train/imdb_train_neg.txt", 'r' ).read()
    nlines = neg_lines.split("\n")   
    nlines = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in nlines]
    nlines = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in nlines]
    
    for line in nlines:
        data = remove_stopwords(line, stopwords)
        indices.append(i)
        text.append(data)
        rating.append("0")
        i = i + 1

    # merge posivite and negative reviews in a single file with their class labels    
    Dataset = list(zip(indices,text,rating))
    
    np.random.shuffle(Dataset)

    df = pd.DataFrame(data = Dataset, columns=['row_Number', 'text', 'polarity'])
    
    # save to csv file
    df.to_csv("dd.csv", index=False, header=True)


def remove_stopwords(sentence, stopwords):
    sentencewords = sentence.split()
    resultwords  = [word for word in sentencewords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result


def unigram_features(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    features = vectorizer.fit_transform(data)
    return features


def bigram_features(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(ngram_range=(1,2))
    features = vectorizer.fit_transform(data)
    return features

def tfidf_features(data):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True,stop_words='english')
    features = vectorizer.fit_transform(data)
    return features

def load_preprocessed_data():
    import pandas as pd 
    data = pd.read_csv("dd.csv",header=0)
    features = data['text'].fillna(' ')
    target = data['polarity']
    return features, target
        

In [4]:
# Preprocessing
print ("Preprocessing...")
preprocessing()
print ("Done.")


Preprocessing...
Done.


In [5]:
print ("Loading Preprocessed data ...")
[features, target] = load_preprocessed_data()

Loading Preprocessed data ...


In [6]:
# Extract Uni-Gram features
uni_features = unigram_features(features)

In [7]:
# Extract Bi-Gram features
bi_features = bigram_features(features)

In [8]:
# Extract TFIDF features
tf_features = tfidf_features(features)

In [9]:
# Logistic Regression using Uni-Gram Features

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build a Logistic Regression model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(uni_features, target, test_size=0.3)

model_log = LogisticRegression(solver='lbfgs')
model_log.fit(features_train, target_train)
predicted = model_log.predict(features_test)

# show prediction results
print("Results using Logistic Regression (Uni-Gram Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Logistic Regression (Uni-Gram Features):
Accuracy 0.8798044878915796
Precision: 0.8801261829652997
Recall: 0.8765709156193896
F Score: 0.8783449516527997


In [10]:
# Logistic Regression using BiGram Features

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build a Logistic Regression model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(bi_features, target, test_size=0.3)

model_log = LogisticRegression(solver='lbfgs')
model_log.fit(features_train, target_train)
predicted = model_log.predict(features_test)

# show prediction results
print("Results using Logistic Regression (BiGram Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Logistic Regression (BiGram Features):
Accuracy 0.8935792046211953
Precision: 0.8823004904146232
Recall: 0.9020054694621695
F Score: 0.8920441739914357


In [11]:
# Logistic Regression using TFIDF Features

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build a Logistic Regression model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(tf_features, target, test_size=0.3)

model_log = LogisticRegression(solver='lbfgs')
model_log.fit(features_train, target_train)
predicted = model_log.predict(features_test)

# show prediction results
print("Results using Logistic Regression (TFIDF Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Logistic Regression (TFIDF Features):
Accuracy 0.8886914019106865
Precision: 0.8688455008488964
Recall: 0.9142474318892363
F Score: 0.890968443960827


In [12]:
# Naive Bayes classification using Uni-Gram Features

from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build a Naive Bayes model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(uni_features, target, test_size=0.3)

model_nb = MultinomialNB()
model_nb.fit(features_train, target_train)
predicted = model_nb.predict(features_test)

# show prediction results
print("Results using Naive Bayes (Uni-Gram Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Naive Bayes (Uni-Gram Features):
Accuracy 0.8631415241057543
Precision: 0.8747637051039697
Recall: 0.840599455040872
F Score: 0.857341361741547


In [13]:
# Naive Bayes classification using Bi-Gram Features

from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build a Naive Bayes model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(bi_features, target, test_size=0.3)

model_nb = MultinomialNB()
model_nb.fit(features_train, target_train)
predicted = model_nb.predict(features_test)

# show prediction results
print("Results using Naive Bayes (Bi-Gram Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Naive Bayes (Bi-Gram Features):
Accuracy 0.8900244390135525
Precision: 0.9017153453871117
Recall: 0.8729802513464991
F Score: 0.887115165336374


In [14]:
# Naive Bayes classification using TFIDF Features

from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Build aNaive Bayes model
# Split the dataset into two sets: 70% Training and 30% Testing
features_train, features_test, target_train, target_test = train_test_split(tf_features, target, test_size=0.3)

model_nb = MultinomialNB()
model_nb.fit(features_train, target_train)
predicted = model_nb.predict(features_test)

# show prediction results
print("Results using Naive Bayes (TFIDF Features):")

# Print Accuracy
print("Accuracy", metrics.accuracy_score(target_test,predicted))

# Print Precision
print("Precision:",metrics.precision_score(target_test, predicted))

# Print Recall
print("Recall:",metrics.recall_score(target_test, predicted))

# Print F score
print("F Score:",metrics.f1_score(target_test, predicted))

Results using Naive Bayes (TFIDF Features):
Accuracy 0.8649189069095756
Precision: 0.8861480075901328
Recall: 0.8354203935599285
F Score: 0.860036832412523
